In [27]:
import os
 
def file_name(file_dir): 
    for root, dirs, files in os.walk(file_dir):
        #print(root) #当前目录路径
        #print(dirs) #当前路径下所有子目录
        print(files) #当前路径下所有非目录子文件
    return files


## 控制c 只有需要检测的图片

In [28]:
c=file_name("testdata")

['baby_GT.bmp', 'bird_GT.bmp', 'butterfly_GT.bmp', 'head_GT.bmp', 'woman_GT.bmp']


In [29]:
for i in c:
    print(i)

baby_GT.bmp
bird_GT.bmp
butterfly_GT.bmp
head_GT.bmp
woman_GT.bmp


In [30]:
import os
filename = os.path.join("testdata/"+c[0])

In [31]:
count=len(c)
print(count)

5


# 加载模型

In [32]:
import argparse

import torch
import torch.backends.cudnn as cudnn
import numpy as np
import PIL.Image as pil_image

from temodel import SRCNN
from myutils import rgb2ycbcr, ycbcr2rgb, calc_psnr

In [33]:
cudnn.benchmark = True
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [34]:
model = SRCNN().to(device)

In [35]:
# state_dict = model.state_dict(torch.load(mydir))

state_dict = model.state_dict()

# 定义psnr函数

In [36]:
def get_psnr(filename,myscale):
    scale=myscale
    image = pil_image.open(filename).convert('RGB')
    image_width = (image.width // scale) * scale
    image_height = (image.height // scale) * scale
    image = image.resize((image_width, image_height), resample=pil_image.BICUBIC)
    image = image.resize((image.width // scale, image.height // scale), resample=pil_image.BICUBIC)
    image = image.resize((image.width *  scale, image.height * scale), resample=pil_image.BICUBIC)
    image.save(filename.replace('.', '_bicubic_x{}.'.format(scale)))
    image = np.array(image).astype(np.float32)
    ycbcr = rgb2ycbcr(image)

    y = ycbcr[..., 0]
    y /= 255.
    y = torch.from_numpy(y).to(device)
    y = y.unsqueeze(0).unsqueeze(0)

    with torch.no_grad():
        preds = model(y).clamp(0.0, 1.0)

    psnr = calc_psnr(y, preds)

    print('PSNR: {:.2f}'.format(psnr))

    preds = preds.mul(255.0).cpu().numpy().squeeze(0).squeeze(0)

    output = np.array([preds, ycbcr[..., 1], ycbcr[..., 2]]).transpose([1, 2, 0])
    output = np.clip(ycbcr2rgb(output), 0.0, 255.0).astype(np.uint8)
    output = pil_image.fromarray(output)
    output.save(filename.replace('.', '_srcnn_x{}.'.format(scale)))
    
    return psnr


# 单独计算psnr

In [37]:
modelname='./model/best.pth'
print(modelname)

./model/best.pth


In [38]:
myscale = 2

In [39]:
for n, p in torch.load(modelname, map_location=lambda storage, loc: storage).items():
        if n in state_dict.keys():
            state_dict[n].copy_(p)
        else:
            raise KeyError(n)

model.eval()
sum_psnr=0
for i in range(count):
    filename = os.path.join("testdata/"+c[i])
    sum_psnr += get_psnr(filename,myscale)

print("psnr is :{:.2f}".format(sum_psnr/count))

C:\Users\Administrator\AppData\Local\Temp\ipykernel_9860\2626010551.py:6: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  image = image.resize((image_width, image_height), resample=pil_image.BICUBIC)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9860\2626010551.py:7: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  image = image.resize((image.width // scale, image.height // scale), resample=pil_image.BICUBIC)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9860\2626010551.py:8: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  image = image.resize((image.width *  scale, image.height * scale), resample=pil_image.BICUBIC)


PSNR: 41.15
PSNR: 38.21
PSNR: 29.52
PSNR: 41.39
PSNR: 34.61
psnr is :36.97


In [16]:
for n, p in torch.load(modelname, map_location=lambda storage, loc: storage).items():
        if n in state_dict.keys():
            state_dict[n].copy_(p)
        else:
            raise KeyError(n)

model.eval()
sum_psnr=0
for i in range(count):
    filename = os.path.join("testdata/"+c[i])
    sum_psnr += get_psnr(filename,myscale)

print("psnr is :{:.2f}".format(sum_psnr/count))

PSNR: 35.47
PSNR: 33.39
PSNR: 25.81
PSNR: 38.94
PSNR: 30.05
psnr is :32.73
